# Imports

In [1]:
import pandas as pd
import numpy as np
from lifelines import KaplanMeierFitter
from fastprogress import master_bar, progress_bar

---

# Carga de datos

In [2]:
auctions = pd.read_pickle("../../../../data/tp2/auctions_tp2_formateado.pkl")

---

In [3]:
labels_v1 = pd.read_pickle("../../targets/targets_ventana_1_auctions.pkl")

In [4]:
auctions_en_v1 = auctions.loc[auctions["device_id"].isin(labels_v1.index)]

In [13]:
gb = auctions_en_v1.groupby("device_id")

In [62]:
vacio = pd.Series()

In [63]:
i = 0
for t, g in gb:
    vacio = vacio.append(g["date"].sub(g["date"].shift()))
    i += 1
    print("{} persona agregada".format(i))
    if (i == 100):
        break

1 persona agregada
2 persona agregada
3 persona agregada
4 persona agregada
5 persona agregada
6 persona agregada
7 persona agregada
8 persona agregada
9 persona agregada
10 persona agregada
11 persona agregada
12 persona agregada
13 persona agregada
14 persona agregada
15 persona agregada
16 persona agregada
17 persona agregada
18 persona agregada
19 persona agregada
20 persona agregada
21 persona agregada
22 persona agregada
23 persona agregada
24 persona agregada
25 persona agregada
26 persona agregada
27 persona agregada
28 persona agregada
29 persona agregada
30 persona agregada
31 persona agregada
32 persona agregada
33 persona agregada
34 persona agregada
35 persona agregada
36 persona agregada
37 persona agregada
38 persona agregada
39 persona agregada
40 persona agregada
41 persona agregada
42 persona agregada
43 persona agregada
44 persona agregada
45 persona agregada
46 persona agregada
47 persona agregada
48 persona agregada
49 persona agregada
50 persona agregada
51 person

In [64]:
vacio = vacio.to_frame()

In [66]:
vacio["en_segundos"] = vacio[0].dt.total_seconds()

In [65]:
vacio = vacio.dropna()

In [67]:
model = KaplanMeierFitter()
model.fit(vacio["en_segundos"])

<lifelines.KaplanMeierFitter: fitted with 10090 observations, 0 censored>

In [68]:
resultados = model.conditional_time_to_event_

# Pruebo con 10.000 individuos, tiempos de todas las ventanas. Desupes probar con solo v1

In [243]:
sampled_targets = labels_v1

In [277]:
ventana_1 = auctions.loc[auctions["dia"] < 21]

In [280]:
labels_v1["n_apariciones"] = ventana_1.groupby("device_id").agg({"dia" : "count"})

In [283]:
muy_aparecidos = labels_v1.loc[labels_v1["n_apariciones"] > 3000]

In [ ]:
len(muy_aparecidos)

In [ ]:
predicciones = []
for j in muy_aparecidos.iterrows():
    mini = auctions.loc[auctions["device_id"] == j[0]] #1677316392823000836
    tiempos = (mini["tiempo"] - mini["tiempo"].shift()).fillna(0)
    model = KaplanMeierFitter()
    model.fit(tiempos)
    resutlados = model.conditional_time_to_event_
    ultima_observacion = mini.loc[(mini["dia"] < 20)]["date"].max()
    sobrevivio = (pd.Timestamp("2019-04-20 00:00:00.000000") - ultima_observacion).total_seconds()
    prediccion = resutlados.loc[resutlados.index > sobrevivio].iloc[0][0]
    predicciones.append(prediccion)
    

In [304]:
len(predicciones)

64

In [305]:
deltas = muy_aparecidos["target"].iloc[:64].values - predicciones

In [306]:
total = 0
for x in deltas:
    total += (x**2)

In [307]:
sqrt(total/64)

28842.80969620438